In [36]:
import os
import numpy as np
import cv2
import random
import glob
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam


In [37]:
train_images = [f for f in glob.glob(r'C:\Users\Bibin\OneDrive\Desktop\camera_gender\Train' + "/**/*", recursive=True) if not os.path.isdir(f)]
random.shuffle(train_images)

In [38]:
train_data=[]
gender_label=[]
for img in train_images:
    image=cv2.imread(img)
    image=cv2.resize(image,(96,96))
    image=img_to_array(image)
    train_data.append(image)

    label = img.split(os.path.sep)[-2]
    if label == "Female" or label=="Female1":
        label = 1
    else:
        label = 0
    
    gender_label.append([label])


In [39]:
train_data = np.array(train_data, dtype="float") / 255.0
gender_label = np.array(gender_label)

In [40]:
(X_train, X_test, Y_train, Y_test) = train_test_split(train_data, gender_label, test_size=0.2,random_state=38)

Y_train = to_categorical(Y_train, num_classes=2)
Y_test = to_categorical(Y_test, num_classes=2)

In [41]:
alter = ImageDataGenerator(rotation_range=25, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode="nearest")

In [42]:
def model_creation():
    model = Sequential()
    

    if backend.image_data_format() == "channels_first":
        image_shape = (3,96,96)
        flag = 1
    else:
        image_shape = (96,96,3)
        flag = -1

    model.add(Conv2D(32, (3,3), padding="same", input_shape=image_shape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=flag))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=flag))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=flag))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=flag))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=flag))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(2))
    model.add(Activation("sigmoid"))

    return model

model=model_creation()

In [46]:
optimizer = Adam(learning_rate=1e-3, weight_decay=(1e-3)/100)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

H = model.fit_generator(alter.flow(X_train, Y_train, batch_size=64),validation_data=(X_test,Y_test), steps_per_epoch=len(X_train) // 64, epochs=100, verbose=1)

model.save('gender.model')

C:\Users\Bibin\AppData\Local\Temp\ipykernel_16564\3606573460.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(alter.flow(X_train, Y_train, batch_size=64),validation_data=(X_test,Y_test), steps_per_epoch=len(X_train) // 64, epochs=100, verbose=1)


Epoch 1/100
155/155 [==============================] - 134s 811ms/step - loss: 0.6695 - accuracy: 0.7146 - val_loss: 0.8879 - val_accuracy: 0.4676
Epoch 2/100
155/155 [==============================] - 141s 911ms/step - loss: 0.5361 - accuracy: 0.7615 - val_loss: 1.3173 - val_accuracy: 0.4688
Epoch 3/100
155/155 [==============================] - 132s 853ms/step - loss: 0.4704 - accuracy: 0.7831 - val_loss: 0.5613 - val_accuracy: 0.7072
Epoch 4/100
155/155 [==============================] - 131s 844ms/step - loss: 0.4371 - accuracy: 0.8049 - val_loss: 0.3932 - val_accuracy: 0.8256
Epoch 5/100
155/155 [==============================] - 124s 803ms/step - loss: 0.3929 - accuracy: 0.8215 - val_loss: 0.3751 - val_accuracy: 0.8437
Epoch 6/100
155/155 [==============================] - 139s 898ms/step - loss: 0.3589 - accuracy: 0.8407 - val_loss: 0.3059 - val_accuracy: 0.8864
Epoch 7/100
155/155 [==============================] - 129s 828ms/step - loss: 0.3463 - accuracy: 0.8498 - val_loss: 0

INFO:tensorflow:Assets written to: gender.model\assets
